In [60]:
import pandas as pd
import os
import csv

In [61]:
# Set filename
db_filename = "wikidata" + os.path.sep + "wikidb_parsed.csv"

# Load the DataFrame
wiki_df = pd.read_csv(db_filename) 

In [62]:
letterpath = 'letterdata'

letterpaths = [os.path.join(letterpath, f) for f in os.listdir(letterpath)]

dfs = []

# Get a list of DataFrames from CSV's
for path in letterpaths:
    try:
        dfs.append(pd.read_csv(path, index_col=None, header=None, encoding='utf-8'))
    except UnicodeDecodeError:
        dfs.append(pd.read_csv(path, index_col=None, header=None, encoding='windows-1252'))

# Concatenate all together
suomi_df = pd.concat(dfs, axis=0, ignore_index=True)

In [63]:
suomi_df.columns = ['title', 'taiv']

In [64]:
# Do an INNER JOIN between suomi sanakirjat and wiktionary words
df = pd.merge(suomi_df, wiki_df, how='inner', on=['title']).drop(['taiv', 'orig_index'],axis=1)

# Force lower case
df["title"] = df["title"].str.lower()

# Drop any duplicate words
df.drop_duplicates(subset=['title'], inplace=True)

# Drop words longer than 14 characters. Most are too difficult for games.
df = df[df['title'].map(len) < 14]

# Reset the index
df = df.reset_index(drop=True)

In [65]:
df[df['is_subs']].sample(30)

,title,is_verb,is_subs,is_adje,is_nume,is_name
10378,kissapeto,False,True,False,False,False
26957,symboliikka,False,True,False,False,False
3845,hallitsija,False,True,False,False,False
32629,viiniköynnös,False,True,False,False,False
23061,riiputus,False,True,False,False,False
17412,nujakointi,False,True,False,False,False
7602,jätti,False,True,False,False,False
9033,kauppahalli,False,True,False,False,False
34176,ärähdys,False,True,False,False,False
8181,kalmukki,False,True,False,False,False


In [67]:
df.tail()

,title,is_verb,is_subs,is_adje,is_nume,is_name
34343,öykkäri,False,True,False,False,False
34344,öykkärimäinen,False,False,True,False,False
34345,öykkäröidä,True,False,False,False,False
34346,öykkäröinti,False,True,False,False,False
34347,öylätti,False,True,False,False,False


In [68]:
# df.index.name = 'id'

In [69]:
from sqlalchemy import create_engine

In [70]:
engine = create_engine('sqlite:///corpus.db', echo=True)
conn = engine.connect()

2020-09-05 16:32:22,620 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-09-05 16:32:22,621 INFO sqlalchemy.engine.base.Engine ()
2020-09-05 16:32:22,623 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-09-05 16:32:22,623 INFO sqlalchemy.engine.base.Engine ()


In [71]:
tablename = "words"

df.to_sql(tablename, conn, if_exists='replace', index_label='id')

2020-09-05 16:32:23,086 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("words")
2020-09-05 16:32:23,087 INFO sqlalchemy.engine.base.Engine ()
2020-09-05 16:32:23,088 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("words")
2020-09-05 16:32:23,088 INFO sqlalchemy.engine.base.Engine ()
2020-09-05 16:32:23,089 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-09-05 16:32:23,090 INFO sqlalchemy.engine.base.Engine ()
2020-09-05 16:32:23,090 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("words")
2020-09-05 16:32:23,091 INFO sqlalchemy.engine.base.Engine ()
2020-09-05 16:32:23,094 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2020-09-05 16:32:23,095 INFO sqlalchemy.engine.base.Engine ('words',)
2020-09-05 16:32:23,096 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list("words")
2020

In [77]:
engine.execute("SELECT * FROM words").fetchall()

2020-09-05 16:34:24,484 INFO sqlalchemy.engine.base.Engine SELECT * FROM words
2020-09-05 16:34:24,484 INFO sqlalchemy.engine.base.Engine ()


[(0, 'aakkonen', 0, 1, 0, 0, 0),
 (1, 'aakkosellinen', 0, 0, 1, 0, 0),
 (2, 'aakkostaa', 1, 0, 0, 0, 0),
 (3, 'aakkosto', 0, 1, 0, 0, 0),
 (4, 'aakkostus', 0, 1, 0, 0, 0),
 (5, 'aalloittainen', 0, 0, 1, 0, 0),
 (6, 'aallokas', 0, 0, 1, 0, 0),
 (7, 'aallokko', 0, 1, 0, 0, 0),
 (8, 'aallonharja', 0, 1, 0, 0, 0),
 (9, 'aallonmurtaja', 0, 1, 0, 0, 0),
 (10, 'aallonpituus', 0, 1, 0, 0, 0),
 (11, 'aallonpohja', 0, 1, 0, 0, 0),
 (12, 'aallota', 1, 0, 0, 0, 0),
 (13, 'aallotar', 0, 1, 0, 0, 0),
 (14, 'aallotus', 0, 1, 0, 0, 0),
 (15, 'aaloe', 0, 1, 0, 0, 0),
 (16, 'aalto', 0, 1, 0, 0, 0),
 (17, 'aaltoalue', 0, 1, 0, 0, 0),
 (18, 'aaltoenergia', 0, 1, 0, 0, 0),
 (19, 'aaltoilla', 1, 0, 0, 0, 0),
 (20, 'aaltoilu', 0, 1, 0, 0, 0),
 (21, 'aaltolevy', 0, 1, 0, 0, 0),
 (22, 'aaltoliike', 0, 1, 0, 0, 0),
 (23, 'aaltopahvi', 0, 1, 0, 0, 0),
 (24, 'aaltopelti', 0, 1, 0, 0, 0),
 (25, 'aaltopituus', 0, 1, 0, 0, 0),
 (26, 'aaltosulje', 0, 1, 0, 0, 0),
 (27, 'aaltosulku', 0, 1, 0, 0, 0),
 (28, 'aaltoviiva'